# **NETTOYAGE DE LA BASE DE DONNÉE DES MANUELS SCOLAIRES**

In [15]:
import pandas as pd
from pathlib import Path
import re

# Configuration des chemins
BASE_DIR = Path('..')
RAW_DIR = BASE_DIR / 'data' / 'raw' / 'Article'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed' / 'final'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [16]:
# Lecture du fichier CSV
input_file = RAW_DIR / 'manuel_scolaire.csv'
df = pd.read_csv(input_file, sep=';')
df.head()  # Aperçu brut

,CLASSE,LIVRES,PRIX
0,Maternelle 1ère année,Langage : Mon cahier d’activités de langage 1è...,1 000
1,Maternelle 1ère année,Mathématiques : Mon cahier d’activités de math...,1 000
2,Maternelle 1ère année,"Dessin, Peinture, Coloriage : Je pratique le d...",1 000
3,Maternelle 2ème année,Langage : Les Majors en activité de langage Ma...,1 000
4,Maternelle 2ème année,Mathématiques : Mon cahier d’activités de math...,1 000


In [17]:
print(f"Nombre de lignes: {len(df)}")
print(f"\nColonnes: {list(df.columns)}")

Nombre de lignes: 645

Colonnes: ['CLASSE', 'LIVRES', 'PRIX']


In [18]:
# Correction de casse
def corriger_casse(texte):
    if pd.isna(texte):
        return texte
    texte = str(texte)
    mots = texte.split()
    return ' '.join([mot.capitalize() if not mot.isupper() else mot for mot in mots])

df['LIVRES'] = df['LIVRES'].apply(corriger_casse)
print("\nAprès correction de la casse:")
print(df['LIVRES'].head())


Après correction de la casse:
0    Langage : Mon Cahier D’activités De Langage 1è...
1    Mathématiques : Mon Cahier D’activités De Math...
2    Dessin, Peinture, Coloriage : Je Pratique Le D...
3    Langage : Les Majors En Activité De Langage Ma...
4    Mathématiques : Mon Cahier D’activités De Math...
Name: LIVRES, dtype: object


In [19]:
# Définition de l'ordre pédagogique
ordre_pedagogique = [
    # Section Francophones
    'Maternelle 1ère année',
    'Maternelle 2ème année', 
    'SIL',
    'CP',
    'CE I',
    'CE II', 
    'CM I',
    'CM II',
    '6e', '5e', '4e', '3e', '2de', '1re', 'Tle',
    
    # Section Anglophones
    'Nursery One',
    'Nursery Two',
    'CLASS I', 'CLASS II', 'CLASS III', 'CLASS IV', 'CLASS V', 'CLASS VI',
    'FORM I', 'FORM II', 'FORM III', 'FORM IV', 'FORM V',
    'LOWER SIXTH',
    'UPPER SIXTH'
]

In [20]:
# Extraction du niveau
def extraire_niveau(classe_text):
    if pd.isna(classe_text):
        return None
    classe_text = str(classe_text).strip()
    for niveau in ordre_pedagogique:
        if niveau.lower() in classe_text.lower():
            return niveau
    return classe_text

df['NIVEAU'] = df['CLASSE'].apply(extraire_niveau)
print("\nNiveaux extraits:")
print(df['NIVEAU'].value_counts())


Niveaux extraits:
NIVEAU
Tle                      141
FORM I                    64
1re                       62
4e                        60
3e                        60
2de                       60
CLASS I                   44
UPPER SIXTH               26
CE I                      22
CLASS V                   20
CM I                      20
FORM V                    16
5e                        12
SIL                       11
CP                        11
6e                         4
Maternelle 2ème année      3
Nursery One                3
Nursery Two                3
Maternelle 1ère année      3
Name: count, dtype: int64


In [21]:
# Création d’un mapping pour l’ordre de tri
ordre_mapping = {niveau: i for i, niveau in enumerate(ordre_pedagogique)}

def get_ordre_tri(niveau):
    return ordre_mapping.get(niveau, len(ordre_pedagogique))

df['ORDRE_TRI'] = df['NIVEAU'].apply(get_ordre_tri)

In [22]:
# Tri final
df_sorted = df.sort_values(['ORDRE_TRI', 'CLASSE', 'LIVRES']).reset_index(drop=True)
print("\nAprès tri pédagogique:")
print(df_sorted[['CLASSE', 'NIVEAU', 'LIVRES']].head(10))


Après tri pédagogique:
                  CLASSE                 NIVEAU  \
0  Maternelle 1ère année  Maternelle 1ère année   
1  Maternelle 1ère année  Maternelle 1ère année   
2  Maternelle 1ère année  Maternelle 1ère année   
3  Maternelle 2ème année  Maternelle 2ème année   
4  Maternelle 2ème année  Maternelle 2ème année   
5  Maternelle 2ème année  Maternelle 2ème année   
6                    SIL                    SIL   
7                    SIL                    SIL   
8                    SIL                    SIL   
9                    SIL                    SIL   

                                              LIVRES  
0  Dessin, Peinture, Coloriage : Je Pratique Le D...  
1  Langage : Mon Cahier D’activités De Langage 1è...  
2  Mathématiques : Mon Cahier D’activités De Math...  
3  Dessin, Peinture, Coloriage : Je Pratique Le D...  
4  Langage : Les Majors En Activité De Langage Ma...  
5  Mathématiques : Mon Cahier D’activités De Math...  
6  Anglais : LIVRET D’ACTIVIT

In [23]:
# Génération du code unique
def generer_code(CLASSE, titre):
    base = (CLASSE[:3].upper() if isinstance(CLASSE, str) else 'XXX')
    identifiant = abs(hash(titre)) % 10000
    return f"{base}_{identifiant}"

df_sorted['code'] = df_sorted.apply(lambda row: generer_code(row['CLASSE'], row['LIVRES']), axis=1)

# Renommage final
df_sorted['designation'] = df_sorted['LIVRES']
df_sorted['type_article'] = 'Livre'

In [24]:
# Nettoyage des prix (en entiers)
def nettoyer_prix(val):
    if pd.isna(val):
        return 0
    val = str(val)
    val = re.sub(r'[^\d]', '', val)
    try:
        return int(val)
    except ValueError:
        return 0

df_sorted['prix'] = df_sorted['PRIX'].apply(nettoyer_prix)

In [25]:
# === AJOUT DES COLONNES POUR STRUCTURE GLOBALE ===
df_final['nature'] = 'Homologué'  
df_final['ville'] = None          # Pas de ville (prix identique dans tout le Cameroun)

In [26]:
# Sélection finale des colonnes
colonnes_finales = ['code', 'designation', 'prix', 'type_article', 'nature', 'classe', 'ville']
df_final = df_final[colonnes_finales]

print("\nDataFrame final:")
print(df_final.head())
print(f"\nNombre de lignes finales: {len(df_final)}")


DataFrame final:
       code                                        designation  prix  \
0   MAT_645  Dessin, Peinture, Coloriage : Je Pratique Le D...  1000   
1  MAT_5216  Langage : Mon Cahier D’activités De Langage 1è...  1000   
2  MAT_9498  Mathématiques : Mon Cahier D’activités De Math...  1000   
3   MAT_645  Dessin, Peinture, Coloriage : Je Pratique Le D...  1000   
4  MAT_4169  Langage : Les Majors En Activité De Langage Ma...  1000   

  type_article     nature                 classe ville  
0       manuel  Homologué  Maternelle 1ère année  None  
1       manuel  Homologué  Maternelle 1ère année  None  
2       manuel  Homologué  Maternelle 1ère année  None  
3       manuel  Homologué  Maternelle 2ème année  None  
4       manuel  Homologué  Maternelle 2ème année  None  

Nombre de lignes finales: 645


In [27]:
print(f"Colonnes : {list(df_final.columns)}")

Colonnes : ['code', 'designation', 'prix', 'type_article', 'nature', 'classe', 'ville']


In [28]:
# Export final
output_file = PROCESSED_DIR / 'manuel_final.csv'
df_final.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n✅ Fichier sauvegardé: {output_file}")


✅ Fichier sauvegardé: ../data/processed/final/manuel_final.csv


In [29]:
francophones = df_final[df_final['classe'].str.contains('Maternelle|SIL|CP|CE|CM|6e|5e|4e|3e|2de|1re|Tle|Terminale', na=False)]
anglophones = df_final[df_final['classe'].str.contains('Nursery|CLASS|FORM|SIXTH', na=False)]

print(f"\nSection Francophone ({len(francophones)} livres):")
francophones.head()


Section Francophone (469 livres):


,code,designation,prix,type_article,nature,classe,ville
0,MAT_645,"Dessin, Peinture, Coloriage : Je Pratique Le D...",1000,manuel,Homologué,Maternelle 1ère année,None
1,MAT_5216,Langage : Mon Cahier D’activités De Langage 1è...,1000,manuel,Homologué,Maternelle 1ère année,None
2,MAT_9498,Mathématiques : Mon Cahier D’activités De Math...,1000,manuel,Homologué,Maternelle 1ère année,None
3,MAT_645,"Dessin, Peinture, Coloriage : Je Pratique Le D...",1000,manuel,Homologué,Maternelle 2ème année,None
4,MAT_4169,Langage : Les Majors En Activité De Langage Ma...,1000,manuel,Homologué,Maternelle 2ème année,None


In [30]:
print(f"\nSection Anglophone ({len(anglophones)} livres):")
anglophones.head()


Section Anglophone (176 livres):


,code,designation,prix,type_article,nature,classe,ville
469,NUR_5837,Drawing And Coloring: INNOVATIVE DRAWING AND C...,1000,manuel,Homologué,Nursery One,None
470,NUR_8907,Language: LANGUAGE ACTIVITY BOOK NURSERY 1 (AT...,1000,manuel,Homologué,Nursery One,None
471,NUR_4206,Mathematics: ELEMENTARY MATHEMATICS ACTIVITY B...,1000,manuel,Homologué,Nursery One,None
472,NUR_6050,Drawing And Coloring: ANUCAM DRAWING AND COLOU...,1000,manuel,Homologué,Nursery Two,None
473,NUR_4752,Language: LANGUAGE ACTIVITY BOOK NURSERY 2 (AT...,1000,manuel,Homologué,Nursery Two,None
